# Install selenium
Selenium은 웹사이트 테스트를 위한 도구로 브라우저 동작을 자동화할 수 있습니다. 셀레니움을 이용하는 웹크롤링 방식은 바로 이점을 적극적으로 활용하는 것입니다. 프로그래밍으로 브라우저 동작을 제어해서 마치 사람이 이용하는 것 같이 웹페이지를 요청하고 응답을 받아올 수 있습니다.

In [7]:
from selenium import webdriver
options = webdriver.ChromeOptions() 
options.add_argument('--headless') 
options.add_argument('--no-sandbox') 
options.add_argument('--disable-dev-shm-usage') 

# Import Library
BeautifulSoup는 HTML과 XML 파일로부터 데이터를 뽑아내기 위한 파이썬 라이브러리입니다.  
urllib HTML에 연결하기 위한 라이브러리입니다.  
pandas dataframe을 사용하기 위한 라이브러리입니다.

# 전역변수
크룰링에 필요한 전역변수들 입니다.  
아래의 랭킹뉴스 - 댓글 많은 순 링크에서 기사 및 댓글 크룰링을 진행했습니다.  
https://news.naver.com/main/ranking/popularMemo.nhn?date=20210308

**date**  
- 검색할 날짜  

In [9]:
TARGET_URL = 'https://news.naver.com/main/ranking/popularMemo.nhn?date=20210316'

TITLE_OF_ARTICLE = []
OFFICE_OF_ARTICLE = []
URL_OF_ARTICLE = []
DATE_OF_ARTICLE = []
CONTENT_OF_ARTICLE = []
COMMENT_OF_ARTICLE = []

# Crawling article & comment
**get_link_from_news_title**  
랭킹 페이지에 존재하는 기사들의 타이틀, 신문사, 본문기사 URL을 가져오는 함수
------------------
- req : 요청을 보낼때, requests 로 보내게 되면 몇몇 사이트에서는 기계가 요청하는것으로 인식해서 응답을 안해주고 차단하는 경우가 있다. 즉, requests로 접근하는 것을 비정상적인 접근으로 보고 차단을 하게 되는것이다. 따라서, 이런 경우에는 requests를 호출 할때 User-Agent 를 지정해서 크롬 브라우저에서의 요청인것으로 인식하게 만들어 오류를 해결할 수 있다.
- source_code_from_URL: 크룰링을 하기 위해 지정된 URL에 연결을 요청한다.
- soup: 데이터를 가져오기 위한 BeautifulSoup Library  
------------------
- for title in soup.find_all('div', {'class': 'ranking_text'}): 페이지에서 보이는 기사들의 제목에 하나씩 접근한다.
- title_link: 기사 제목이 위치하고 있는 태그를 가리킨다.
- comment_link: 기사의 모든 댓글들을 보기위한 URL이 있는 태그를 가리킨다.
- article_URL: 기사의 본문이 존재하는 URL을 저장한다.
- comment_URL: 기사의 댓글이 존재하는 URL을 저장한다.
- TITLE_OF_ARTICLE.append: 기사 제목을 리스트에 저장한다.
- OFFICE_OF_ARTICLE.append: 기사의 신문사를 리스트에 저장한다.
- URL_OF_ARTICLE.append: 기사의 URL을 리스트에 저장한다.
- get_text: get_text 함수에 article_URL을 넘겨준다.
- get_comment: get_comment 함수에 comment_URL을 넘겨준다.

In [ ]:
def get_link_from_news_title(URL):
    req = urllib.request.Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    source_code_from_url = urllib.request.urlopen(req)
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
    
    for title in soup.select('div.list_content'): #child(1) 을 of-type(1)
        url_link = title.select('a')
        article_URL = 'https://news.naver.com' + url_link[0]['href']
        URL_OF_ARTICLE.append(article_URL)

        position = article_URL.index('read.nhn?')
        comment_URL = article_URL[:position+9] + 'm_view=1&includeAllCount=true&' + article_URL[position+9:] 

        get_text(article_URL) 
        get_comment(comment_URL) #코맨트는 더보기를 눌러야 가져온다...

**get_link_from_news_title**    
위의 함수에서 받은 기사 본문 URL에 접근하여서 기사를 가져오는 함수  

--------------

- req : 요청을 보낼때, requests 로 보내게 되면 몇몇 사이트에서는 기계가 요청하는것으로 인식해서 응답을 안해주고 차단하는 경우가 있다. 즉, requests로 접근하는 것을 비정상적인 접근으로 보고 차단을 하게 되는것이다. 따라서, 이런 경우에는 requests를 호출 할때 User-Agent 를 지정해서 크롬 브라우저에서의 요청인것으로 인식하게 만들어 오류를 해결할 수 있다.
- source_code_from_URL: 크룰링을 하기 위해 지정된 URL에 연결을 요청한다.
- soup: 데이터를 가져오기 위한 BeautifulSoup Library   
- date_of_article: 기사의 작성된 날짜가 있는 태그를 가리킨다.
- DATE_OF_ARTICLE.append: 리스트에 기사의 날짜를 저장한다.
- string_item: 문단이 나뉘어져 있는 경우가 있어서 나뉜 기사들을 하나로 합쳐서 저장할 변수
- for item in soup.select('div._article_body_contents'): 나뉘어져 있는 모든 기사의 문단에 하나씩 접근한다.
- string: 태그를 제외한 string 형식으로 된 기사들을 가져온다.
- string_tiem: 기사들을 하나로 만든다.
- CONTENT_OF_ARTICLE.append: 기사의 내용을 리스트에 저장한다.

In [ ]:
 def get_text(article_URL):
    req = urllib.request.Request(article_URL, headers={'User-Agent': 'Mozilla/5.0'})
    source_code_from_url = urllib.request.urlopen(req)
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
    
    #office_link = soup.select('head > meta:nth-of-type(6)')
    office_link = soup.select('div.press_logo > a > img')
    OFFICE_OF_ARTICLE.append(office_link[0]['title'])

    title_link = soup.select('div.article_info > h3')
    TITLE_OF_ARTICLE.append(title_link[0].string)

    date_of_article = soup.select('span.t11')
    DATE_OF_ARTICLE.append(date_of_article[0].string)
    
    string_item = '' #빈 문자열 만드는 것
    for item in soup.select('div._article_body_contents'):  
        string = str(item.find_all(text=True))
        string_item += string #  string_item = string_item + string 눈에 보이는 문단 구분이 아니라, 다른 문단 구분이 있을수 있다..html로 봤을때...
    CONTENT_OF_ARTICLE.append(string_item) 

**get_comment**  
위의 함수에서 받은 기사 댓글 URL에 접근하여서 URL을 가져오는 함수

--------------

- driver: 동작할 웹드라이버를 크롬으로 설정합니다.  
- driver.implicitly_wait: 페이지 로딩이 완전히 이루어질 때까지 기다리기 위해 삽입하였습니다. 인터넷 속도가 빠르시다면 줄이셔도 됩니다.
- driver.get: 기사 댓글 URL에 연결합니다.

- contents: 댓글이 존재하는 태그의 위치를 가리킨다.
- comments: 댓글을 저장할 리스트
- for content in contents: 댓글에 하나씩 접근하는 반복문입니다.
- comments.append: 댓글을 리스트에 저장한다.
- COMMENT_OF_ARTICLE.append: 위에 comments로 만들어진 리스트를 저장하는 리스트.

---------------

- while True: 반복문 break를 만나기 전까지 무한히 반복한다.
- try: 댓글의 더보기를 더이상 누를수 없을때 까지 반복한다.
- more_button: 더보기 버튼이 위치하는 태그를 가리킨다.
- more_button.click(): 더보기 버튼을 클릭한다.
- time.sleep(1): 페이지가 로딩되는데 시간이 필요하므로 1초의 텀을 준다.
- except: 댓글을 더이상 누를수 없을때 실행된다.
- break: 반복문을 빠져나온다. 


In [ ]:
def get_comment(comment_URL):
    driver = webdriver.Chrome('chromedriver', options=options)
    driver.implicitly_wait(5)
    driver.get(comment_URL) #URL 창을 여는 것

    # while True:
    #     try:
    #         more_button = driver.find_element_by_css_selector('a.u_cbox_btn_more')
    #         more_button.click()
    #         time.sleep(1)
    #     except:
    #         break

    # html = driver.page_source
    # soup = BeautifulSoup(html, 'html.parser')   
    # comments = []
    # for comment in soup.select('div.u_cbox_text_wrap > span.u_cbox_contents'): 
    #    comments.append(comment.get_text())  

    contents = driver.find_elements_by_css_selector('span.u_cbox_contents')
    comments = []
    for comment in contents:
        comments.append(comment.text) #BS4에서는 .string이었고 셀 에서는 .text

    COMMENT_OF_ARTICLE.append(comments) #댓글일 다 붙어버리면 시작 끝 알수 없으니, 리스트에 리스트로 구분 가능하게 

# Main
위에서 작성한 함수들을 실행한다.

In [ ]:
start = time.time()
get_link_from_news_title(TARGET_URL)
print("time :", time.time() - start)

time : 1348.6139223575592


# DataFrame
리스트들을 합쳐서 하나의 dataframe으로 생성한다.

In [ ]:
df = pd.DataFrame(list(zip(DATE_OF_ARTICLE, TITLE_OF_ARTICLE, OFFICE_OF_ARTICLE, CONTENT_OF_ARTICLE, URL_OF_ARTICLE, COMMENT_OF_ARTICLE)), columns =['Date', 'Title', 'Office', 'Content', 'URL', 'Comment']) 

In [ ]:
df.head()

,Date,Title,Office,Content,URL,Comment
0,2021.03.08. 오후 1:40,"""정말정말 죄송하다"" 몸 낮춘 민주당…박영선 ""수사 착수해야""",JTBC,"['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n', '""정말정말 죄송하다"" 몸 낮춘 민주당…박영선 ""수사 착수해야""', ' ', '@img_tag_s@', '〈YONHAP PHOTO-2410〉 최고위원회의에서 발언하는 이낙연 대표 (서울=연합뉴스) 하사헌 기자 = 더불어민주당 이낙연 대표가 5일 국회에서 열린 최고위원회의에서 발언하고 있다. 2021.3.5 toadboy@yna.co.kr/2021-03-05 10:10:04/ 〈저작권자 ⓒ 1980-2021 ㈜연합뉴스. 무단 전재 재배포 금지.〉', '@img_tag_e@', ' ""최근 LH 공사 직원의 투기 의혹으로 시민께서 얼마나 큰 분노와 실망을 느끼고 계실지 저희들도 아프도록 잘 압니다. 정말 송구스럽다는 말씀을 올립니다""(이낙연 더불어민주당 대표) ', ' ', "" 더불어민주당이 한국토지주택공사(LH) 직원들의 '3기 신도시 부동산 투기 의혹'을 비판하고 3월 국회 내에 재발 방지 대책을 마련하겠다고 밝혔습니다. 이낙연 더불어민주당 대표는 이번 의혹에 대해 또다시 사과한 뒤 철저한 조사와 처벌에 나서겠다며 이같이 말했습니다. "", ' ', ' 이 대표는 오늘(8일) 오전 4·7 서울시장 보궐선거 박영선 후보 캠프에서 진행된 중앙선대위 첫 회의에서 ""가장 강력하게 응징하고 재발 방지 대책을 최단 시일 내에 수립해 다시는 이런 일이 없도록 확실히 하겠다""고 말했습니다. ', ' ', ' 특히 이 대표는 ""총리실 주도 조사가 진행돼 며칠 안에 1차 결과가 발표될 예정이다""며 ""가족이나 친인척을 포함한 가명·차명 계좌는 강제 수사를 해서라도 밝혀내고, 현행법이 허용하는 가장 강력한 처벌을 하겠다""고 강조했습니다. 그러면서 ""앞으로 강제수사 과정을 시민사회와 협력해 수사하는 체제로 임할 것을 약속드리겠다. 그렇게 해서 한 점 의심을 남기지 않도록 최선을 다하겠다""고 밝혔습니다. ', ' ', ' 이날 민주당은 공직자 투기 사건이 재발하지 않도록 입법에 나서겠단 계획도 내놨습니다. 김태년 원내대표는 ""이른바 \'한국토지주택공사(LH) 투기 방지법\'을 3월 국회의 최우선 처리 법안으로 추진하겠다""고 밝혔습니다. 김 원내대표는 ""금융 범죄와 마찬가지로 공직자의 부동산 투기 이익을 환수하겠다""며 ""투기 이익에 3∼5배의 벌금을 부과하는 내용의 법안을 문진석 의원이 발의했고, 박상혁 의원도 발의 예정""이라고 말했습니다. ', ' ', ' 이어 김 원내대표는 ""투기 의혹이 제기된 당 소속 시의원이 윤리감찰단 조사를 앞두고 탈당한 데 대해서는 영구히 복당을 금지하겠다""면서 ""부동산 정책에 대한 당정의 진정성마저 훼손하는 사건으로 대단히 엄중하게 받아들인다. 백 마디 말보다 실질적 행동과 결과를 보여드리겠다""고 강조했습니다. ', ' ', ' 회의에 참석한 박영선 더불어민주당 서울시장 후보도 ""불법으로 얻은 이익은 남김없이 환수하고 몰수해야 한다""고 주장했습니다. 박 후보는 ""(이번 의혹과 관련해) 철저하게 전면조사하는 것은 물론이고 강력하게 수사에 착수해 단 한점의 의문 없이 규명해야 한다""고 말했습니다. 그러면서 ""만약 제가 서울시장이 된다면 서울시청과 SH 주택공사에 이와 유사한 일이 절대 일어나지 않도록 철저하게 제도를 만들겠다""고 강조했습니다. ', ' ', ' 한편 당 상임선거대책위원장을 맡은 이 대표는 이날 박 후보에 대해서는 ""오랜 기간 서울시장을 꿈꾸고 설계해왔고, 구상과 전략을 동시에 갖춘 아주 보기 드문 지도자""라며 ""박 후보에게 서울시를 맡겨줄 것을 간곡히 부탁드린다""고 지지를 호소했습니다. ', '김필준 기자 (kim.piljun@jtbc.co.kr)', '▶ 뉴스의 뒷이야기! JTBC 취재썰', '▶ 시청자와 함께! JTBC 뉴스 제보하기', '▶ 관점과 분석이 있는 새로운 뉴스룸', 'Copyright by JTBC(https://jtbc.joins.com) All Rights Reserved. 무단 전재 및 재배포 금지\n\t', ' // 본문 내용 ', '\n']",https://news.naver.com/main/ranking/read.nhn?mode=LSD&mid=shm&sid1=001&oid=437&aid=0000260823&rankingType=RANKING,"[이럴려구 180석 줬냐 무능한 민주당 다시는 상종 안한다국민들이 바보인거지 한심하다, 김필준기자야, 낱말이란 것이 나만 사용하라고 있는 것도 아니고 너만 사용하라고 있는 것도 아니다. 그런데 낱말을 선택해서 사용할 때에 누군가 들으면 몹시 기분나쁜 낱말이거나 남을 화나게 하는 낱말은 사용하지 마라, 어짜피 똥싼당빨갱이들이 HK해서 늬들 DS 시켜줄거니까 걱정 없다 이거냐?, 필요할때만 몸 낯추고 멋대로하면서, 기냥 신도시 취소허구 공무원들 짜름 되여, 수사 결과는 이미 나와있겠구먼...박 정권때 이미 시작된 투기현 정권에 터졌음. 따라서 현 정권은 아주 잘못 없음 ! 천벌을 받을...., 변창흠: 그낭 모르고 땅을샀는데 신도시로 지정됨.계란열사: 그냥 계란을 던져봤는데 이낙연이 튀어나옴., 눈가리고 아웅 연기도 잘하고 철판도 두껍고, 철저한 조사?처벌?말로만?처벌은 중형으로 한다는 말이 없네.또 그냥 넘어가겠네., 무지하게 똥줄이 타긴 타나부네 ㅎㅎㅎ 표심 얻을라고 그렇게 세금 퍼주더니 엉뚱한데서 터졌네, 좌이티비씨 좌파넘들이 잘못한 기사는 정치란에 올려서 댓글공감순으로 배치 못하게 했네ㅋㅋ 이게 어딜봐서 정치기사냐, 사과는 하지만 표는 달라??, 제대로 안밝히면 고위공직자들 중에도 저짓거리한 종자들이 많다는거다!! 전수조사 해서 전원 재산 몰수해라!!, 더듬어 만진당과 문재앙 정부는 답이없다!, 잡아서 제산 환수하고 공무원 짤라라, 좌파 총리실 수사착수...설문지 배포? 자진신고 ㅋ 애들 코미디언...군 카톡 휴가 이후 웃긴다. ㅋㅋ ㅋ, 트위터 cho : 싹싹 빌때 족쳐야 한다]"
1,2021.03.08. 오전 11:36,"윤석열, 사퇴 후 이재명 제쳤다…대권 적합도 '1위'",JTBC,"['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n', '14.6%→32.4% 지지율 상승…이재명 지사 제치고 선두', ' ', '@img_tag_s@', '〈사진-연합뉴스〉', '@img_tag_e@', ' 윤석열 전 검찰총장이 차기 대선후보 적합도 1위라는 여론조사 결과가 나왔습니다. ', ' ', ' 사퇴 후 처음 이뤄진 여론조사에서 이재명 경기도지사를 제치고 선두에 올랐습니다. ', ' ', ' 오늘(8일) 여론조사 기관 한국사회여론연구소(KSOI)은 TBS 의뢰로 지난 5일 차기 대선후보 적합도를 조사한 결과를 발표했습니다. ', ' ', ' 조사 결과, 윤 전 총장은 32.4%로 가장 높은 지지를 받았습니다. ', ' ', ' 그다음은 이재명 지사가 24.1%, 이낙연 대표가 14.9%로 나타났습니다. ', ' ', ' 무소속 홍준표 의원(7.6%), 정세균 국무총리(2.6%), 추미애 전 법무부 장관(2.5%)이 뒤를 이었습니다. ', ' ', ' ', '@img_tag_s@', '〈사진-한국사회여론연구소(KSOI)〉', '@img_tag_e@', ' ', '■14.6%→32.4% 지지율 수직 상승', ' ', ' ', ' 지난 1월 22일 같은 기관에서 실시된 여론조사에서 윤 전 총장의 지지율은 14.6%였습니다. ', ' ', ' 당시 윤 전 총장은 이재명 지사와 이낙연 대표에 이어 적합도 3위로 조사됐습니다. ', ' ', ' 이번엔 17.8%p나

In [ ]:
df.to_csv("df.csv",index=True, encoding='utf-8-sig')